In [1]:
import pandas as pd
import numpy as np
import pickle
import os
from collections import Counter

import sys
sys.path.append("/home/ualtintas/bluegill/")
import bluegill.run as bgrun
import bluegill.plt as bgplt

In [2]:

BED = pd.read_table('data/CRE.bed', names=['Chr', 'Start', 'End', 'Name'])

# >>> 1 >>>
BWS = []
for signal in ["ATAC","H3K27ac", "H3K4me3","RNAseq", 'AR', 'FOXA1']:
    for h in ["Vh","30m","4h","16h","72h"]:
        BWS += [f"/groups/lackgrp/projects/col-matt-LNCaP/SIGNAL/{signal}_{h}.bw"]
# <<< 1 <<<

#/groups/lackgrp/projects/col-matt-LNCaP/SIGNAL/

In [3]:
%%bash

#path=/groups/lackgrp/projects/col-matt-LNCaP/ProcessedDataSeo/ATAC.BED
#parallel cat $path/AT{}* '|' "awk '{if (\$9 > 10) print \$0}' " '|' cut -f1-3 '|' sort -k1,1 -k2,2n '|' bedtools merge -i - '>' data/ATAC.{}.bed  ::: Vh 30m 4h 16h 72h

cat data/ATAC.Vh.bed data/ATAC.30m.bed data/ATAC.4h.bed data/ATAC.16h.bed data/ATAC.72h.bed \
| cut -f1-3 \
| sort -k1,1 -k2,2n \
| bedtools merge -i - \
| bedtools intersect -a - -b /groups/lackgrp/genomeAnnotations/hg19/hg19-blacklist.v2.bed -v \
| grep -v '-' | grep -v '_' \
>  data/ATAC_ALL.raw.bed

bedtools intersect -a data/ATAC_ALL.raw.bed -b data/CRE.bed  -wa -wb \
| cut -f1-3,7  > data/ATAC_CRE.raw.bed

In [6]:
BED = pd.read_table('data/ATAC_CRE.raw.bed', names=['Chr', 'Start', 'End', 'Name'])

Nbins=1
h=1000

BED, samples = bgrun.runSignal(BED, BWS, "data/Epigenome.p",  Nbins=Nbins, h=h, nP=100)

0: ATAC_Vh.bw :: 1: ATAC_Vh.bw :: 2: ATAC_Vh.bw :: 3: ATAC_Vh.bw :: 4: ATAC_Vh.bw :: 5: ATAC_Vh.bw :: 6: ATAC_Vh.bw :: 7: ATAC_Vh.bw :: 8: ATAC_Vh.bw :: 9: ATAC_Vh.bw :: 10: ATAC_Vh.bw :: 11: ATAC_Vh.bw :: 12: ATAC_Vh.bw :: 13: ATAC_Vh.bw :: 14: ATAC_Vh.bw :: 15: ATAC_Vh.bw :: 16: ATAC_Vh.bw :: 17: ATAC_Vh.bw :: 18: ATAC_Vh.bw :: 19: ATAC_Vh.bw :: 20: ATAC_Vh.bw :: 21: ATAC_Vh.bw6: ATAC_30m.bw :: 7: ATAC_30m.bw :: 3: ATAC_30m.bw :: 5: ATAC_30m.bw4: ATAC_30m.bw :: 0: ATAC_30m.bw22: ATAC_Vh.bw ::  ::  ::  :: 23: ATAC_Vh.bw8: ATAC_30m.bw9: ATAC_30m.bw ::  ::  :: 24: ATAC_Vh.bw1: ATAC_30m.bw ::  :: 10: ATAC_30m.bw2: ATAC_30m.bw11: ATAC_30m.bw25: ATAC_Vh.bw ::  ::  ::  :: 26: ATAC_Vh.bw12: ATAC_30m.bw ::  :: 13: ATAC_30m.bw :: 27: ATAC_Vh.bw :: 14: ATAC_30m.bw28: ATAC_Vh.bw ::  :: 29: ATAC_Vh.bw :: 30: ATAC_Vh.bw :: 16: ATAC_30m.bw :: 17: ATAC_30m.bw :: 31: ATAC_Vh.bw :: 32: ATAC_Vh.bw18: ATAC_30m.bw15: ATAC_30m.bw ::  ::  :: 19: ATAC_30m.bw20: ATAC_30m.bw33: ATAC_Vh.bw ::  ::  :: 34: ATAC_

In [7]:


S = pickle.load(open("data/Epigenome.p","rb"))
N = np.concatenate((
    bgrun.runTMM(S[:,0:5,:])[:,:,0],
    bgrun.runTMM(S[:,5:10,:])[:,:,0],
    bgrun.runTMM(S[:,10:15,:])[:,:,0],
    bgrun.runTMM(S[:,15:20,:])[:,:,0],
    bgrun.runTMM(S[:,20:25,:])[:,:,0],
    bgrun.runTMM(S[:,25:30,:])[:,:,0]),
    axis=1)
samples = [s.split('/')[-1].split('.')[0] for s in BWS]
N = pd.DataFrame(N, columns=samples, index=BED['Name']).drop_duplicates()
N.to_csv('zenodo/Epigenome.tmm.tsv', sep='\t')
N

,ATAC_Vh,ATAC_30m,ATAC_4h,ATAC_16h,ATAC_72h,H3K27ac_Vh,H3K27ac_30m,H3K27ac_4h,H3K27ac_16h,H3K27ac_72h,...,AR_Vh,AR_30m,AR_4h,AR_16h,AR_72h,FOXA1_Vh,FOXA1_30m,FOXA1_4h,FOXA1_16h,FOXA1_72h
Name,,,,,,,,,,,,,,,,,,,,,
chr1:751985-756985,3.532207,4.163705,4.333421,6.070663,4.077631,11.595370,12.948841,18.910203,17.974253,13.157234,...,9.074192,8.718933,7.795667,9.359501,6.342948,8.050763,6.849383,8.194302,9.188222,6.978041
chr1:760223-765223,53.434340,54.316282,60.112466,55.685252,51.789856,53.898098,51.961280,44.143940,53.174820,86.533809,...,9.788812,17.123498,15.349682,22.885930,16.620331,22.953928,24.885314,31.495199,34.737976,21.881506
chr1:791444-796444,16.733823,14.580213,13.193328,14.272607,15.172989,13.699383,15.757297,14.692529,17.761326,13.262050,...,5.713604,5.660805,7.692565,8.515448,7.602123,5.960414,6.232203,6.731016,5.332528,3.537601
chr1:802847-807847,10.465679,8.852990,9.143239,7.565637,7.227980,1.206438,1.840906,1.602189,1.433826,1.425413,...,10.981824,9.943838,7.754475,12.625446,7.551128,11.566372,6.301133,8.464685,5.764199,7.774548
chr1:863945-868945,39.861734,40.916517,44.197912,46.117635,41.868412,37.678508,39.851433,28.806427,51.972198,29.875828,...,12.886337,10.999054,21.431320,13.842626,19.311345,13.977502,13.128843,14.678849,11.819867,21.678653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
chrY:28496921-28501921,1.866831,1.819342,1.221673,1.313881,1.053899,1.595695,2.425486,1.778686,3.528180,1.774541,...,2.777559,3.581502,6.750092,1.351106,5.052391,3.061573,1.748221,3.722252,1.882767,4.506726
chrY:28508739-28513739,0.286555,0.700169,1.687724,3.537595,1.328305,0.107846,0.431465,0.061705,0.221395,0.000000,...,1.847705,3.632424,7.548258,7.748847,1.549452,1.522007,1.983885,2.070774,2.224210,5.285912
chrY:28534627-28539627,2.053148,1.345728,1.974445,2.429269,2.361556,0.025894,0.152796,0.399441,0.329889,0.213615,...,1.539797,2.401813,1.693471,2.081836,1.647519,2.645544,3.247106,1.329761,2.749259,4.694565


# STAR

In [8]:
BWS = [
    '/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS31_Vehicle-Rep-1.FORWARD.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS31_Vehicle-Rep-1.REVERSE.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS33_30m-DHT-Rep-1.FORWARD.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS33_30m-DHT-Rep-1.REVERSE.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS34_4hr-DHT-Rep-1.FORWARD.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS34_4hr-DHT-Rep-1.REVERSE.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS35_16hr-DHT-Rep-1.FORWARD.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS35_16hr-DHT-Rep-1.REVERSE.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS36_72hr-DHT-Rep-1.FORWARD.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS36_72hr-DHT-Rep-1.REVERSE.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS37_Vehicle-Rep-2.FORWARD.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS37_Vehicle-Rep-2.REVERSE.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS39_30m-DHT-Rep-2.FORWARD.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS39_30m-DHT-Rep-2.REVERSE.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS40_4hr-DHT-Rep-2.FORWARD.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS40_4hr-DHT-Rep-2.REVERSE.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS41_16hr-DHT-Rep-2.FORWARD.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS41_16hr-DHT-Rep-2.REVERSE.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS42_72hr-DHT-Rep-2.FORWARD.bw',
	'/groups/lackgrp/projects/col-matt-LNCaP/results/bigwig/SS42_72hr-DHT-Rep-2.REVERSE.bw'
]

Nbins=1
h=100

BED, samples = bgrun.runSignal(BED, BWS, "data/STAR.p",  Nbins=Nbins, h=h, nP=100, type_='max')

0: SS31_Vehicle-Rep-1.FORWARD.bw :: 1: SS31_Vehicle-Rep-1.FORWARD.bw :: 2: SS31_Vehicle-Rep-1.FORWARD.bw :: 3: SS31_Vehicle-Rep-1.FORWARD.bw :: 4: SS31_Vehicle-Rep-1.FORWARD.bw :: 5: SS31_Vehicle-Rep-1.FORWARD.bw :: 6: SS31_Vehicle-Rep-1.FORWARD.bw :: 7: SS31_Vehicle-Rep-1.FORWARD.bw :: 8: SS31_Vehicle-Rep-1.FORWARD.bw :: 9: SS31_Vehicle-Rep-1.FORWARD.bw :: 10: SS31_Vehicle-Rep-1.FORWARD.bw :: 11: SS31_Vehicle-Rep-1.FORWARD.bw :: 12: SS31_Vehicle-Rep-1.FORWARD.bw :: 13: SS31_Vehicle-Rep-1.FORWARD.bw7: SS31_Vehicle-Rep-1.REVERSE.bw ::  :: 6: SS31_Vehicle-Rep-1.REVERSE.bw4: SS31_Vehicle-Rep-1.REVERSE.bw3: SS31_Vehicle-Rep-1.REVERSE.bw14: SS31_Vehicle-Rep-1.FORWARD.bw2: SS31_Vehicle-Rep-1.REVERSE.bw0: SS31_Vehicle-Rep-1.REVERSE.bw5: SS31_Vehicle-Rep-1.REVERSE.bw ::  ::  ::  ::  :: 8: SS31_Vehicle-Rep-1.REVERSE.bw ::  :: 1: SS31_Vehicle-Rep-1.REVERSE.bw15: SS31_Vehicle-Rep-1.FORWARD.bw9: SS31_Vehicle-Rep-1.REVERSE.bw ::  ::  ::  :: 16: SS31_Vehicle-Rep-1.FORWARD.bw :: 10: SS31_Vehicle-Rep-

In [9]:


S = pickle.load(open("data/STAR.p","rb"))
S = bgrun.runTMM(S)[:,:,0]

F = np.zeros((len(S), 5))
R = np.zeros((len(S), 5))

for i in range(5):
    F[:,i] = (S[:,2*i] + S[:,(2*i)+10]) / 2
    R[:,i] = (S[:,2*i+1] + S[:,(2*i)+11]) / 2



S = pd.DataFrame(np.concatenate((F,R), 1), columns=['Vh_F', '30m_F', '4h_F', '16h_F', '72h_F', 'Vh_R', '30m_R', '4h_R', '16h_R', '72h_R'], index=BED['Name'])

S.to_csv('data/STAR.tsv', sep='\t')


